# Baseline GNN model

For the baseline model we will use a fully connected graph, and in the first iteration ignore the edge features (weights). This model should do better than a simple MLP at classifying the nodes.

In [1]:
import pickle
import numpy as np
import torch

from pathlib import Path
from itertools import permutations
from torch_geometric.data import Data, Dataset

In [2]:
INSTANCE_PATH = Path("data")
RESULTS_PATH = Path("results")

In [3]:
with open(INSTANCE_PATH / 'dataset.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
graph = data[0]
graph

(array([[0.5       , 0.5       , 0.        ],
        [0.06505159, 0.94888554, 2.        ],
        [0.96563203, 0.80839735, 4.        ],
        [0.30461377, 0.09767211, 8.        ],
        [0.68423303, 0.44015249, 7.        ],
        [0.12203823, 0.49517691, 9.        ],
        [0.03438852, 0.9093204 , 8.        ],
        [0.25877998, 0.66252228, 5.        ],
        [0.31171108, 0.52006802, 2.        ],
        [0.54671028, 0.18485446, 5.        ],
        [0.96958463, 0.77513282, 8.        ]]),
 array([-1,  0,  1,  2,  1,  0,  0,  0,  2,  2,  1]))

For the graph is fully connected and the edge features (representing the distance between each neighbour) is ignored.

It's not yet clear how the the demand capacities of the vehicles will be modeled.

In [5]:
x = torch.tensor(graph[0])
y = torch.tensor(graph[1] + 1).view((-1, 1))
edge_indices = torch.tensor(list(permutations(np.arange(x.size(0)), r=2)), dtype=torch.long).t().contiguous()

In [6]:
x.shape, y.shape, edge_indices.shape

(torch.Size([11, 3]), torch.Size([11, 1]), torch.Size([2, 110]))

In [7]:
def get_data(graph):
    x = torch.tensor(graph[0], dtype=torch.float)
    y = torch.tensor(graph[1] + 1).view((-1, 1))
    edge_indices = torch.tensor(list(permutations(np.arange(x.size(0)), r=2)), dtype=torch.long).t().contiguous()
    
    return Data(x=x, edge_index=edge_indices, y=y)

In [8]:
get_data(graph)

Data(x=[11, 3], edge_index=[2, 110], y=[11, 1])

In [9]:
class VRPDataset(Dataset):
    def __init__(self, raw_data):
        super().__init__()
        self.graph_data = self._process(raw_data)
        
    def _process(self, raw_data):
        return [get_data(d) for d in raw_data]
        
    def len(self):
        return len(self.graph_data)

    def get(self, index):
        return self.graph_data[index]

In [10]:
dataset = VRPDataset(data)

In [11]:
dataset, dataset[0]

(VRPDataset(2000), Data(x=[11, 3], edge_index=[2, 110], y=[11, 1]))